# В данном ноутбуке собирается база ссылок на вакансии
Глобальная логика такая. Для начала формируется общий список профессий, которые мы бы хотели собрать. После этого формируются поисковые запросы на сайте HeadHanters. Дальше со страницы собираются все найденные ссылки на вакансии. Каждая страница на HeadHanters имеет отдельный url, поэтому есть возможность собрать вообще все ссылки, которые связанны с этой профессией.

Для начала требуется создать функцию, которая бы транслитировала русский текст в английский

In [3]:
import re

def iteritems(dic):
    return iter(dic.items())

def transliterate(string):

    capital_letters = {u'А': u'A',
                       u'Б': u'B',
                       u'В': u'V',
                       u'Г': u'G',
                       u'Д': u'D',
                       u'Е': u'E',
                       u'Ё': u'E',
                       u'З': u'Z',
                       u'И': u'I',
                       u'Й': u'Y',
                       u'К': u'K',
                       u'Л': u'L',
                       u'М': u'M',
                       u'Н': u'N',
                       u'О': u'O',
                       u'П': u'P',
                       u'Р': u'R',
                       u'С': u'S',
                       u'Т': u'T',
                       u'У': u'U',
                       u'Ф': u'F',
                       u'Х': u'H',
                       u'Ъ': u'',
                       u'Ы': u'Y',
                       u'Ь': u'',
                       u'Э': u'E',}

    capital_letters_transliterated_to_multiple_letters = {u'Ж': u'Zh',
                                                          u'Ц': u'Ts',
                                                          u'Ч': u'Ch',
                                                          u'Ш': u'Sh',
                                                          u'Щ': u'Sch',
                                                          u'Ю': u'Yu',
                                                          u'Я': u'Ya',}


    lower_case_letters = {u'а': u'a',
                       u'б': u'b',
                       u'в': u'v',
                       u'г': u'g',
                       u'д': u'd',
                       u'е': u'e',
                       u'ё': u'e',
                       u'ж': u'zh',
                       u'з': u'z',
                       u'и': u'i',
                       u'й': u'y',
                       u'к': u'k',
                       u'л': u'l',
                       u'м': u'm',
                       u'н': u'n',
                       u'о': u'o',
                       u'п': u'p',
                       u'р': u'r',
                       u'с': u's',
                       u'т': u't',
                       u'у': u'u',
                       u'ф': u'f',
                       u'х': u'h',
                       u'ц': u'ts',
                       u'ч': u'ch',
                       u'ш': u'sh',
                       u'щ': u'sch',
                       u'ъ': u'',
                       u'ы': u'y',
                       u'ь': u'',
                       u'э': u'e',
                       u'ю': u'yu',
                       u'я': u'ya',}

    capital_and_lower_case_letter_pairs = {}

    for capital_letter, capital_letter_translit in iteritems(capital_letters_transliterated_to_multiple_letters):
        for lowercase_letter, lowercase_letter_translit in iteritems(lower_case_letters):
            capital_and_lower_case_letter_pairs[u"%s%s" % (capital_letter, lowercase_letter)] = u"%s%s" % (capital_letter_translit, lowercase_letter_translit)

    for dictionary in (capital_and_lower_case_letter_pairs, capital_letters, lower_case_letters):

        for cyrillic_string, latin_string in iteritems(dictionary):
            string = re.sub(cyrillic_string, latin_string, string)

    for cyrillic_string, latin_string in iteritems(capital_letters_transliterated_to_multiple_letters):
        string = re.sub(cyrillic_string, latin_string.upper(), string)

    return string

Дальше определяем основные функции, которые нам потребуются - создание url, сбор данных с определенной страницы, сбор данных по всем страницам

In [4]:
import requests
from bs4 import BeautifulSoup

In [9]:
def get_url(string):
    return "https://hh.ru/vacancies/" + transliterate(string)

def get_url_page(string, page):
    return "https://hh.ru/vacancies/" + transliterate(string) + '?page=' + str(page)


#сбор данных одной страницы
def get_vacancies_url(url):
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}

    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text)
    return list(map(lambda x: x.get('href'), soup.findAll('a', {'class' : 'bloko-link HH-LinkModifier'})))


#сбор данных со всех страниц
def get_vacancies_urls(search):
    print('\n' + search)
    res = get_vacancies_url(get_url(search))
    print(get_url(search))

    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}

    r = requests.get(get_url(search), headers=headers)
    soup = BeautifulSoup(r.text)
    counter = 0
    soup_for_np = soup.findAll('a', {'class' : 'bloko-button HH-Pager-Control'})
    while len(soup_for_np) < 2:
        counter += 1
        if counter > 10:
            return search
        soup_for_np = soup.findAll('a', {'class' : 'bloko-button HH-Pager-Control'})
    num_pages = int(soup_for_np[-1].text)
    for i in range(1, num_pages):
        print(get_url_page(search, i))
        loc_res = get_vacancies_url(get_url_page(search, i))
        if (len(loc_res) > 0):
            for el in loc_res:
                if not el in res:
                    res.append(el)
        else:
            print('No Info')
    return res
    

В следующей ячейке идет создание базовых поисковых запросов и приведение их к виду, который можно перевести в url.
Функция транслита не всегда корректно работает (в случае внедрения в бизнес процессы это будет исправленно), поэтому некоторые запросы изменяются вручную

In [7]:
prof = 'Работник торгового зала, Руководитель отдела продаж, Региональный менеджер, komplektovshik, Администратор, Товаровед, Кавист, Пекарь, Кондитер, Оператор, Супервайзер, Бариста, Бармен, tekhnolog, dispetcher, elektromontazhniki, formovshik, frezerovshik, markirovshik, mekhanik, specialist_po_zakupkam, stropalshik, svarshik'
profs = prof.split(sep = ', ')
profs = list(map(lambda x: x.replace(' ', '_'), profs))

В следующей ячейке и происходит весь сбор и сохранение данных в отдельную папку. Работает долго, поэтому запускать не рекомендуется. Все данные уже собраны и лежат в папке url_vacancy

In [10]:
def save(name, array):
    f = open('url_vacancy\\' + name, 'w')
    for url in array:
        f.write(url + '\n')
    f.close()


for prof in profs:
    name = prof.lower()
    urls = get_vacancies_urls(name)
    if urls == name:
        bad_vac.append(name)
    else:
        name = transliterate(name)
        save(name, urls)
        print(len(urls))


tekhnolog
https://hh.ru/vacancies/tekhnolog
https://hh.ru/vacancies/tekhnolog?page=1
https://hh.ru/vacancies/tekhnolog?page=2
https://hh.ru/vacancies/tekhnolog?page=3
https://hh.ru/vacancies/tekhnolog?page=4
https://hh.ru/vacancies/tekhnolog?page=5
https://hh.ru/vacancies/tekhnolog?page=6
https://hh.ru/vacancies/tekhnolog?page=7
https://hh.ru/vacancies/tekhnolog?page=8
https://hh.ru/vacancies/tekhnolog?page=9
https://hh.ru/vacancies/tekhnolog?page=10
https://hh.ru/vacancies/tekhnolog?page=11
https://hh.ru/vacancies/tekhnolog?page=12
https://hh.ru/vacancies/tekhnolog?page=13
https://hh.ru/vacancies/tekhnolog?page=14
https://hh.ru/vacancies/tekhnolog?page=15
https://hh.ru/vacancies/tekhnolog?page=16
https://hh.ru/vacancies/tekhnolog?page=17
https://hh.ru/vacancies/tekhnolog?page=18
741

dispetcher
https://hh.ru/vacancies/dispetcher
https://hh.ru/vacancies/dispetcher?page=1
https://hh.ru/vacancies/dispetcher?page=2
https://hh.ru/vacancies/dispetcher?page=3
https://hh.ru/vacancies/dispetc